<a href="https://colab.research.google.com/github/eujkim/DS2002/blob/main/Finance_API_Homework_withoutapikey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#### PROFESSOR WILLIAMSON GRANTED ME AN EXTENSION IN CLASS ON 2/26/25 THROUGH THE WEEKEND.

In [1]:
import json
import pandas as pd
import json
import requests

In [ ]:
### Task 1: Fetch Basic Stock Data

In [2]:
## input two or more stock symbols
stock=input()

AAPL,MSFT


In [3]:
## Use the API to fetch data for the given symbol

apikey=""

url = "https://yfapi.net/v6/finance/quote"

querystring = {"symbols":stock}

headers = {
    'x-api-key': apikey
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

print(json.dumps(response.json(), indent=4))

{"quoteResponse":{"result":[{"language":"en-US","region":"US","quoteType":"EQUITY","typeDisp":"Equity","quoteSourceName":"Nasdaq Real Time Price","triggerable":true,"customPriceAlertConfidence":"HIGH","currency":"USD","shortName":"Apple Inc.","longName":"Apple Inc.","hasPrePostMarketData":true,"firstTradeDateMilliseconds":345479400000,"priceHint":2,"marketState":"CLOSED","corporateActions":[],"postMarketTime":1740790793,"regularMarketTime":1740776402,"exchange":"NMS","messageBoardId":"finmb_24937","exchangeTimezoneName":"America/New_York","exchangeTimezoneShortName":"EST","gmtOffSetMilliseconds":-18000000,"market":"us_market","esgPopulated":false,"regularMarketChangePercent":1.9131873,"regularMarketPrice":241.84,"postMarketChangePercent":-0.0951024,"postMarketPrice":241.61,"postMarketChange":-0.229996,"regularMarketChange":4.5399933,"regularMarketDayHigh":242.09,"regularMarketDayRange":"234.51 - 242.09","regularMarketDayLow":234.51,"regularMarketVolume":56522103,"regularMarketPreviousC

In [4]:
df = pd.DataFrame(response.json()['quoteResponse']['result'])
df.head()

,language,region,quoteType,typeDisp,quoteSourceName,triggerable,customPriceAlertConfidence,currency,shortName,longName,...,marketCap,forwardPE,priceToBook,sourceInterval,exchangeDataDelayedBy,averageAnalystRating,tradeable,cryptoTradeable,displayName,symbol
0,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,USD,Apple Inc.,Apple Inc.,...,3632944644096,29.102285,54.493010,15,0,2.0 - Buy,False,False,Apple,AAPL
1,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,USD,Microsoft Corporation,Microsoft Corporation,...,2951215579136,26.554514,9.751179,15,0,1.4 - Strong Buy,False,False,Microsoft,MSFT


In [5]:
## Extract Stock Ticker, Company Name, and Current Market Price
df['regularMarketPrice'] = df['regularMarketPrice'].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A")


## Display the results
print(df[['symbol', 'longName', 'regularMarketPrice']])

  symbol               longName regularMarketPrice
0   AAPL             Apple Inc.            $241.84
1   MSFT  Microsoft Corporation            $396.99


In [ ]:
### Task 2.1: Fetch Additional Data Using Modules

In [6]:
## Module from Quote Summary Endpoint

import requests
import json

apikey=""

url = "https://yfapi.net/v11/finance/quoteSummary/AAPL"

querystring = {
    "modules": "financialData,summaryDetail"
}

headers = {
    'x-api-key': apikey
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

print(json.dumps(response.json(), indent=4))

{"quoteSummary":{"result":[{"summaryDetail":{"maxAge":1,"priceHint":{"raw":2,"fmt":"2","longFmt":"2"},"previousClose":{"raw":237.3,"fmt":"237.30"},"open":{"raw":236.95,"fmt":"236.95"},"dayLow":{"raw":234.51,"fmt":"234.51"},"dayHigh":{"raw":242.09,"fmt":"242.09"},"regularMarketPreviousClose":{"raw":237.3,"fmt":"237.30"},"regularMarketOpen":{"raw":236.95,"fmt":"236.95"},"regularMarketDayLow":{"raw":234.51,"fmt":"234.51"},"regularMarketDayHigh":{"raw":242.09,"fmt":"242.09"},"dividendRate":{"raw":1.0,"fmt":"1.00"},"dividendYield":{"raw":0.0041,"fmt":"0.41%"},"exDividendDate":{"raw":1739145600,"fmt":"2025-02-10"},"payoutRatio":{"raw":0.1571,"fmt":"15.71%"},"fiveYearAvgDividendYield":{"raw":0.59,"fmt":"0.59"},"beta":{"raw":1.2,"fmt":"1.20"},"trailingPE":{"raw":38.326466,"fmt":"38.33"},"forwardPE":{"raw":29.102285,"fmt":"29.10"},"volume":{"raw":56522103,"fmt":"56.52M","longFmt":"56,522,103"},"regularMarketVolume":{"raw":56522103,"fmt":"56.52M","longFmt":"56,522,103"},"averageVolume":{"raw":50

In [8]:
## Convert it into a Pandas Data Frame.

df = pd.DataFrame(response.json()['quoteSummary']["result"][0])
df.head()

,summaryDetail,financialData
maxAge,1,86400
priceHint,"{'raw': 2, 'fmt': '2', 'longFmt': '2'}",NaN
previousClose,"{'raw': 237.3, 'fmt': '237.30'}",NaN
open,"{'raw': 236.95, 'fmt': '236.95'}",NaN
dayLow,"{'raw': 234.51, 'fmt': '234.51'}",NaN


In [9]:
df['financialData']['returnOnAssets']['fmt']

'22.52%'

In [10]:
## Find 52 Week High, 52 Week Low, and Return on Assets (ROA)

print ("52 week High:" + str(df['summaryDetail']["fiftyTwoWeekHigh"]["raw"]))
print ("52 week Low:" + str(df['summaryDetail']["fiftyTwoWeekLow"]["raw"]))
print ("ROA: " + df['financialData']['returnOnAssets']['fmt'])

52 week High:260.1
52 week Low:164.08
ROA: 22.52%


In [ ]:
### Task 2.2: Fetch Additional Data Using Modules

In [11]:
## Find Current Trending Stocks (Professor Williamson said Top 5 was fine)

import requests
import pandas as pd

apikey=""

url = "https://yfapi.net/v1/finance/trending/US"

headers = {
    'x-api-key': apikey
    }

response = requests.request("GET", url, headers=headers, params=querystring)

df = pd.DataFrame(response.json()['finance']["result"])
df['quotes'][0][3]['symbol']

for n in range(5):
    stock = df['quotes'][0][n]['symbol']
    print(stock)

BTC-USD
XRP-USD
XLM-USD
XRP-CAD
ADA-USD


In [12]:
## Find Their Actual Name, Ticker, Current Price, and 52 high and low
import requests
import pandas as pd

apikey=""

url = "https://yfapi.net/v1/finance/trending/US"

headers = {
    'x-api-key': apikey
    }

response = requests.request("GET", url, headers=headers, params=querystring)

df = pd.DataFrame(response.json()['finance']["result"])
df['quotes'][0][3]['symbol']

for n in range(5):
    stock = df['quotes'][0][n]['symbol']
    print(stock)

url = "https://yfapi.net/v6/finance/quote"

querystring = {"symbols":stock}

headers = {
    'x-api-key': apikey
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

print(json.dumps(response.json(), indent=4))

BTC-USD
XRP-USD
XLM-USD
XRP-CAD
ADA-USD
{"quoteResponse":{"result":[{"language":"en-US","region":"US","quoteType":"CRYPTOCURRENCY","typeDisp":"Cryptocurrency","quoteSourceName":"CoinMarketCap","triggerable":true,"customPriceAlertConfidence":"HIGH","marketState":"REGULAR","exchange":"CCC","messageBoardId":"finmb_ADA_CCC","exchangeTimezoneName":"UTC","exchangeTimezoneShortName":"UTC","gmtOffSetMilliseconds":0,"market":"ccc_market","esgPopulated":false,"shortName":"Cardano USD","currency":"USD","corporateActions":[],"regularMarketTime":1740891420,"regularMarketChangePercent":1.9095423,"regularMarketPrice":0.665128,"longName":"Cardano USD","firstTradeDateMilliseconds":1510185600000,"priceHint":6,"circulatingSupply":35212423168,"lastMarket":"CoinMarketCap","volume24Hr":590259200,"volumeAllCurrencies":590259200,"fromCurrency":"ADA","toCurrency":"USD=X","regularMarketVolume":590259200,"regularMarketPreviousClose":0.6592016,"fullExchangeName":"CCC","regularMarketOpen":0.6592016,"averageDailyVo